# Collaborative Filtering from Scratch

The goal is to implement a collaborative filtering algorithm from scratch. We should have the following progression:
- Basics
- CF algorithm
- Naive collaborative filtering with Alternating Least Squares
- Naive CF with SGD
- Naive CF with streaming updates

Then we can make our matrix CF model non-linear:
- Non-linear CF
- Non-linear CF with streaming

#### Extras
But modern CF models use neural networks...

- Using NN to do basic CF
- NN for non-linear CF
- NN for CF with side data
- Bayesian CF

#### A good flavor of CF: Tag-based CF

If we have time, we can make a more efficient recommendation system that's
- more storage efficient
- good for online training scenario

## Basics

The recommender problem is one of **matrix completion**

We start with a rating matrix $R$, which is $N$ x $M$ ($n$ users, $m$ items)

Let's say our target is the predicted ratings  $\hat{Y}_{ui} \approx R$. Then our optimization problem becomes
$$ \mathcal{L}(Z) = \sum_{ij:Y_{ij \neq ?}} ||Z - Y||^2$$

With some constraints, we can make the problem solvable:
- $Y$ is low rank $\rightarrow Z = UV^T \approx Y$
- We can map $U$ and $V$ pair-wise to latent variables $\rightarrow U$ is [$N$ x $K$], $V$ is [$M$ x $K$]

This also means we need to "fill out" $U$ and $V$.

For basic collaborative filtering with matrices, we can do this with Alternating Least Squares (ALS).

#### Simple ALS

In [188]:
import numpy as np

In [246]:
#2 users, 3 items, 5 latents
n = 2
m = 3
k = 5
noise = 1e-4
#Generates [2,3] array
R = np.array([[0, 1, 0], [1, 1, 0]], dtype=float)

In [247]:
#Alternating Least Squares
best_U = np.random.randn(n,k)
best_Vt = np.random.randn(m,k)

max_iter = 10
#We can do this in closed form

#Solve for V first, by fixing U
best_Vt = np.linalg.inv(best_U.T @ best_U + noise*np.eye(k)) @ best_U.T @ R
#Solve for U, by fixing V
best_U = (np.linalg.inv(best_Vt @ best_Vt.T + noise*np.eye(k)) @ best_Vt @ R.T).T

In [248]:
#Check RMSE
np.sqrt(np.mean((R - (best_U @ best_Vt))**2))

6.826374448653433e-05

This is linear ALS, we can improve our results with non-linear methods like PMF (Probabilistic Matrix Factorization) later.

#### Training and Inference

By using a training dataset, how do we predict on unseen data?

In [ ]:
#TODO: Code here.

#### Probabilistic Matrix Factorization

We can also fit $U$ and $V$ using Probabilistic Matrix Factorization.
- paper: https://proceedings.neurips.cc/paper_files/paper/2007/file/d7322ed717dedf1eb4e6e52a37ea7bcd-Paper.pdf

In [ ]:
#TODO: Code here